In [23]:
import pandas as pd
import numpy as np
from lib import raw_ds_columns, calc_ts_diff
from visualization import display_point_process_events
from scipy.optimize import minimize
from tick.hawkes import HawkesSumExpKern


In [2]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot, row, column, grid
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
ticks_atm_log_mon = pd.read_csv("data/current.csv", header=None, names=raw_ds_columns())
ticks_atm_log_mon = ticks_atm_log_mon.iloc[1:]

ticks_atm_log_mon["Bid_time"] = ticks_atm_log_mon["Bid_time"].astype(int)
ticks_atm_log_mon["Mid_IV"] = ticks_atm_log_mon["Mid_IV"].astype(float)
ticks_atm_log_mon["Mid_price"] = ticks_atm_log_mon["Mid_price"].astype(float)


time_start = min(ticks_atm_log_mon["Bid_time"])
ticks_atm_log_mon["Bid_time"] = ticks_atm_log_mon["Bid_time"] - time_start
ticks_atm_log_mon = ticks_atm_log_mon.sort_values("Bid_time")

In [4]:
ticks_atm_log_mon

,Trade_date,Bid_time,Option_expiration,Call_Put,Price_strike,Bid_price,Bid_size,Ask_price,Ask_size,Mid_price,Underlying_mid_price,log_moneyness,Time_to_expiry,Mid_IV
1.0,10/21/2020,0,12/18/2020,C,3435.0,140.7,12,141.7,79,141.20,3440.56,0.0016173231597010362,0.1589041095890411,0.253263
2.0,10/21/2020,0,12/18/2020,C,3435.0,140.7,12,141.7,79,141.20,3440.56,0.0016173231597010362,0.1589041095890411,0.253263
0.0,10/21/2020,533,12/18/2020,P,3435.0,137.2,12,138.1,108,137.65,3440.56,0.0016173231597010362,0.1589041095890411,0.256945
3.0,10/21/2020,4157,12/18/2020,C,3435.0,140.7,12,141.5,34,141.10,3440.56,0.0016173231597010362,0.1589041095890411,0.253080
4.0,10/21/2020,4157,12/18/2020,C,3435.0,140.7,12,141.5,34,141.10,3440.56,0.0016173231597010362,0.1589041095890411,0.253080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672.0,10/21/2020,27369169,12/18/2020,C,3500.0,96.5,30,97.3,30,96.90,3444.2650000000003,-0.01605243947771865,0.1589041095890411,0.222399
1674.0,10/21/2020,27371475,12/18/2020,P,3440.0,137.2,15,138.0,15,137.60,3444.17,0.0012114751698509452,0.1589041095890411,0.255274
1675.0,10/21/2020,27371475,12/18/2020,P,3440.0,137.2,15,138.0,15,137.60,3444.17,0.0012114751698509452,0.1589041095890411,0.255274
1673.0,10/21/2020,27371476,12/18/2020,C,3440.0,130.6,15,131.5,20,131.05,3444.2200000000003,0.001225992350208193,0.1589041095890411,0.235618


In [5]:
tick_diff_atm = calc_ts_diff(ticks_atm_log_mon, keys=['Mid_price', 'Bid_time', 'Mid_IV'])
tick_diff_atm["Mid_price_ret"] = np.log1p(ticks_atm_log_mon["Mid_price"].pct_change())
tick_diff_atm["Mid_IV_diff_sign"] = np.sign(tick_diff_atm["Mid_IV_diff"])



tick_diff_atm

/home/eugene/PycharmProjects/HPVol/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/eugene/PycharmProjects/HPVol/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Trade_date,Bid_time,Option_expiration,Call_Put,Price_strike,Bid_price,Bid_size,Ask_price,Ask_size,Mid_price,Underlying_mid_price,log_moneyness,Time_to_expiry,Mid_IV,Mid_price_diff,Bid_time_diff,Mid_IV_diff,Mid_price_ret,Mid_IV_diff_sign
2.0,10/21/2020,0,12/18/2020,C,3435.0,140.7,12,141.7,79,141.20,3440.56,0.0016173231597010362,0.1589041095890411,0.253263,0.00,0.0,0.000000,0.000000,0.0
0.0,10/21/2020,533,12/18/2020,P,3435.0,137.2,12,138.1,108,137.65,3440.56,0.0016173231597010362,0.1589041095890411,0.256945,-3.55,533.0,0.003682,-0.025463,1.0
3.0,10/21/2020,4157,12/18/2020,C,3435.0,140.7,12,141.5,34,141.10,3440.56,0.0016173231597010362,0.1589041095890411,0.253080,3.45,3624.0,-0.003865,0.024755,-1.0
4.0,10/21/2020,4157,12/18/2020,C,3435.0,140.7,12,141.5,34,141.10,3440.56,0.0016173231597010362,0.1589041095890411,0.253080,0.00,0.0,0.000000,0.000000,0.0
5.0,10/21/2020,6608,12/18/2020,P,3435.0,137.2,12,138.3,180,137.75,3440.56,0.0016173231597010362,0.1589041095890411,0.257128,-3.35,2451.0,0.004048,-0.024028,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672.0,10/21/2020,27369169,12/18/2020,C,3500.0,96.5,30,97.3,30,96.90,3444.2650000000003,-0.01605243947771865,0.1589041095890411,0.222399,-40.05,12946.0,-0.014955,-0.345936,-1.0
1674.0,10/21/2020,27371475,12/18/2020,P,3440.0,137.2,15,138.0,15,137.60,3444.17,0.0012114751698509452,0.1589041095890411,0.255274,40.70,2306.0,0.032876,0.350671,1.0
1675.0,10/21/2020,27371475,12/18/2020,P,3440.0,137.2,15,138.0,15,137.60,3444.17,0.0012114751698509452,0.1589041095890411,0.255274,0.00,0.0,0.000000,0.000000,0.0
1673.0,10/21/2020,27371476,12/18/2020,C,3440.0,130.6,15,131.5,20,131.05,3444.2200000000003,0.001225992350208193,0.1589041095890411,0.235618,-6.55,1.0,-0.019657,-0.048772,-1.0


In [6]:

def distr_hist(data, bins=200):
    hist, edges = np.histogram(data, density=True, bins=200)
    p = figure(title=data.name)
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
    return p

def count_hist(data, bins):
    p = figure(title="Bid_time", x_axis_label='Bid Time')
    edges = np.linspace(min(data["Bid_time"]), max(data["Bid_time"]), bins)
    hist_plus, hist_minus = [], []
    for i in range(1, len(edges)):
        l, r = edges[i - 1], edges[i]
        bucket_ = data[(data["Bid_time"] >= l) & (data["Bid_time"] <= r)]
        hist_plus.append(len(bucket_[bucket_["Mid_IV_diff_sign"] > 0]))
        hist_minus.append(len(bucket_[bucket_["Mid_IV_diff_sign"] < 0]))
    p.quad(top=hist_plus, bottom=0, left=edges[:-1], right=edges[1:], line_color="white", fill_color="green", fill_alpha=0.6, legend_label="Up tick")
    p.quad(top=hist_minus, bottom=0, left=edges[:-1], right=edges[1:], line_color="white", fill_color="red", fill_alpha=0.6, legend_label="Down tick")
    return p

def mean_hist(data, mean_col, bins):
    p = figure(title=mean_col, x_axis_label='Bid Time')
    edges = np.linspace(min(data["Bid_time"]), max(data["Bid_time"]), bins)
    hist = []
    for i in range(1, len(edges)):
        l, r = edges[i - 1], edges[i]
        hist.append(np.mean(data[(data["Bid_time"] >= l) & (data["Bid_time"] <= r)][mean_col]))
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white", legend_label="{} mean value".format(mean_col))
    return p

def show_ts_df(ts_df):
    p2 = count_hist(ts_df, bins=100)

    p3_a = mean_hist(ts_df, "Mid_price", bins=300)
    p3_b = distr_hist(ts_df["Mid_price_ret"], bins=300)
    
    p4_a = mean_hist(ts_df, "Mid_IV", bins=300)
    p4_b = distr_hist(ts_df["Mid_IV_diff"])
    grid_ = grid(column(row(p2, sizing_mode='stretch_width'), row(p3_a, p3_b, sizing_mode='stretch_width'), row(p4_a, p4_b, sizing_mode='stretch_width')))

    show(grid_)

show_ts_df(tick_diff_atm)

In [7]:
def quantile_analysis(data, columns):
    rows = []
    for col in columns:
        duration_Q99 = np.quantile(data[col], 0.99)
        duration_Q95 = np.quantile(data[col], 0.95)
        duration_Q90 = np.quantile(data[col], 0.90)
        duration_MAX = max(data[col])
        duration_N = len(data[col])
        duration_N_gt_2_Q99 = len(data[col][data[col] > 2*duration_Q99])
        duration_N_gt_Q99 = len(data[col][data[col] > duration_Q99])
        duration_frac = "{:.2f}%".format((duration_N_gt_2_Q99 / duration_N) * 100)
        rows.append([col, duration_Q90, duration_Q95, duration_Q99, duration_MAX, duration_N, duration_N_gt_Q99, duration_N_gt_2_Q99, duration_frac])
    return pd.DataFrame(
        rows, 
        columns=["Name", "Q90", "Q95", "Q99", "Max", "N", "N>Q99", "N>2*Q99", "Fraction"]
    )


In [8]:
quantile_analysis(tick_diff_atm, ["Bid_time_diff", "Mid_price_ret", "Mid_IV_diff"])

,Name,Q90,Q95,Q99,Max,N,N>Q99,N>2*Q99,Fraction
0,Bid_time_diff,44889.500000,84473.000000,216254.500000,3.105038e+06,1676,17,2,0.12%
1,Mid_price_ret,0.077256,0.201977,0.379158,5.557933e-01,1676,17,0,0.00%
2,Mid_IV_diff,0.015207,0.016692,0.030651,4.570552e-02,1676,17,0,0.00%


In [9]:
def filter_gt_Q99(df, columns):
    for col in columns:
        df = df.loc[~(df[col] > np.quantile(df[col], 0.99))]
    return df

def filter_zeros(df, columns):
    for col in columns:
        df = df[df[col] != 0.0]
    return df

def remove_n_first(df, n):
    return df.iloc[n:]

def remove_outliers(data, columns):
    df = data.copy()
    df = filter_gt_Q99(df, columns)
    df = filter_zeros(df, columns)
#     one of the datasets has a supermassive black hole between the ticks
    df = remove_n_first(df, 5)
    return df
        

In [10]:
filtered_df = remove_outliers(tick_diff_atm, ["Bid_time_diff", "Mid_price_ret", "Mid_IV_diff"])
filtered_df

,Trade_date,Bid_time,Option_expiration,Call_Put,Price_strike,Bid_price,Bid_size,Ask_price,Ask_size,Mid_price,Underlying_mid_price,log_moneyness,Time_to_expiry,Mid_IV,Mid_price_diff,Bid_time_diff,Mid_IV_diff,Mid_price_ret,Mid_IV_diff_sign
10.0,10/21/2020,3131954,12/18/2020,C,3500.0,99.4,238,100.0,44,99.70,3441.73,-0.016788716548648823,0.1589041095890411,0.229643,-23.15,17976.0,-0.041286,-0.208798,-1.0
11.0,10/21/2020,3131987,12/18/2020,C,3500.0,99.4,238,100.4,30,99.90,3441.9,-0.016739324004297996,0.1589041095890411,0.229872,0.20,33.0,0.000229,0.002004,1.0
12.0,10/21/2020,3146613,12/18/2020,C,3500.0,100.3,147,101.5,34,100.90,3444.1850000000004,-0.01607566676397418,0.1589041095890411,0.229832,1.00,14626.0,-0.000039,0.009960,-1.0
23.0,10/21/2020,3382080,12/18/2020,P,3435.0,137.3,15,138.2,15,137.75,3445.5550000000003,0.003068068863502391,0.1589041095890411,0.261431,-1.00,1.0,0.017495,-0.007233,1.0
24.0,10/21/2020,3382754,12/18/2020,P,3435.0,137.2,90,138.2,60,137.70,3445.25,0.0029795450988045684,0.1589041095890411,0.261078,-0.05,674.0,-0.000353,-0.000363,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666.0,10/21/2020,27323443,12/18/2020,C,3500.0,96.3,30,97.2,30,96.75,3444.38,-0.01601905119888867,0.1589041095890411,0.222028,-17.55,87095.0,-0.048417,-0.166696,-1.0
1670.0,10/21/2020,27355691,12/18/2020,C,3430.0,136.3,15,137.5,45,136.90,3444.25,0.004145912764722784,0.1589041095890411,0.237281,3.15,35.0,-0.020350,0.023278,-1.0
1671.0,10/21/2020,27356223,12/18/2020,C,3430.0,136.4,15,137.5,15,136.95,3444.2700000000004,0.004151719527278634,0.1589041095890411,0.237353,0.05,532.0,0.000072,0.000365,1.0
1672.0,10/21/2020,27369169,12/18/2020,C,3500.0,96.5,30,97.3,30,96.90,3444.2650000000003,-0.01605243947771865,0.1589041095890411,0.222399,-40.05,12946.0,-0.014955,-0.345936,-1.0


In [11]:
quantile_analysis(filtered_df, ["Bid_time_diff", "Mid_price_ret", "Mid_IV_diff"])

,Name,Q90,Q95,Q99,Max,N,N>Q99,N>2*Q99,Fraction
0,Bid_time_diff,67009.000000,111913.000000,179784.000000,215517.000000,801,8,0,0.00%
1,Mid_price_ret,0.138629,0.244849,0.341536,0.363625,801,8,0,0.00%
2,Mid_IV_diff,0.015629,0.017307,0.026428,0.027837,801,8,0,0.00%


In [12]:
show_ts_df(filtered_df)

In [13]:
n_plus_ts, n_minus_ts = filtered_df.loc[filtered_df["Mid_IV_diff_sign"] > 0], \
                        filtered_df.loc[filtered_df["Mid_IV_diff_sign"] < 0]

In [14]:
n_plus_ts

,Trade_date,Bid_time,Option_expiration,Call_Put,Price_strike,Bid_price,Bid_size,Ask_price,Ask_size,Mid_price,Underlying_mid_price,log_moneyness,Time_to_expiry,Mid_IV,Mid_price_diff,Bid_time_diff,Mid_IV_diff,Mid_price_ret,Mid_IV_diff_sign
11.0,10/21/2020,3131987,12/18/2020,C,3500.0,99.4,238,100.4,30,99.90,3441.9,-0.016739324004297996,0.1589041095890411,0.229872,0.20,33.0,0.000229,0.002004,1.0
23.0,10/21/2020,3382080,12/18/2020,P,3435.0,137.3,15,138.2,15,137.75,3445.5550000000003,0.003068068863502391,0.1589041095890411,0.261431,-1.00,1.0,0.017495,-0.007233,1.0
30.0,10/21/2020,3472675,12/18/2020,C,3435.0,138.3,60,139.1,45,138.70,3445.0,0.0029069787913093108,0.1589041095890411,0.244385,5.60,64945.0,0.001889,0.041213,1.0
31.0,10/21/2020,3472718,12/18/2020,P,3435.0,137.4,60,138.2,60,137.80,3445.0,0.0029069787913093108,0.1589041095890411,0.261047,-0.90,43.0,0.016662,-0.006510,1.0
43.0,10/21/2020,3563397,12/18/2020,P,3435.0,137.3,15,138.0,15,137.65,3445.23,0.0029737399879848827,0.1589041095890411,0.260969,-0.45,11845.0,0.000090,-0.003264,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648.0,10/21/2020,27132296,12/18/2020,P,3435.0,133.7,15,134.6,35,134.15,3445.51,0.003055008472982005,0.1589041095890411,0.254801,-1.15,26.0,0.017137,-0.008536,1.0
1650.0,10/21/2020,27190743,12/18/2020,P,3440.0,136.3,15,137.2,15,136.75,3445.075,0.0014742035254849397,0.1589041095890411,0.254505,5.00,837.0,0.018379,0.037248,1.0
1660.0,10/21/2020,27207733,12/18/2020,P,3440.0,136.8,15,137.7,15,137.25,3444.98,0.0014466275482234278,0.1589041095890411,0.255337,5.50,16451.0,0.019211,0.040898,1.0
1662.0,10/21/2020,27236319,12/18/2020,P,3375.0,114.2,49,114.4,23,114.30,3444.84,0.020482134940651417,0.1589041095890411,0.270444,-22.95,28586.0,0.015107,-0.182978,1.0


In [16]:
display_point_process_events(n_minus_ts)

In [17]:
def get_learner(decays, verbose=False):
    return HawkesSumExpKern(decays, verbose=verbose, tol=1e-10, max_iter=1000)


def get_HP_exp_learner(decays, events, verbose=False):
    learner = get_learner(decays, verbose)
    learner.fit(events)
    if verbose:
        print("Adj: {}, decays: {}, baselines: {}".format(learner.adjacency, decays, learner.baseline))
    return learner


def fun(decays, events):
    return - HawkesSumExpKern(decays=decays, penalty="elasticnet", elastic_net_ratio=0.9, solver="agd", max_iter=1000).fit(events).score()


In [24]:
decays_ = [0.001, 0.01, 0.5]
events_ = np.array(list(n_plus_ts["Bid_time"].astype(float)))
decays_hat = minimize(fun, x0=[decays_], method="Nelder-Mead", args=[events_], tol=1e-05).x
learner = get_HP_exp_learner(decays_hat, [events_])

In [26]:
learner.adjacency

array([[[2.15708190e-01, 5.64286249e-05, 8.26457306e-04]]])